# Web Intelligence
# Text Processing and Similarity Search

#### Prof. Claudio Lucchese

## Similarity search

Similarity search is a fundamental task in Web and Data Mining. It's a building block for:
 - **Plagiarism Detection**. Consider newly published document that may infringe copyrights, including images and videos.
 - **Mirror Pages**. Crawled pages from a search engines may have 40% mirror pages.
 - **Recommendation Systems**. On-line purchases, Movies, Hotels, Restaurants, ...
 - **Person identification**. Personal devices, security monitoring, ...

## Similarity Search in text

Data source:
    https://www.kaggle.com/mousehead/songlyrics

**Task:
Find the most similar song to "Pink" by Aerosmith**


In [1]:
songs_file = "datasets/lyrics/songdata.csv"

In [ ]:
#linux only
!head {songs_file}

In [2]:
# Tentative: read the first 10 lines and inspect the content

import csv
# see DickReader https://docs.python.org/3/library/csv.html

with open(songs_file, newline='') as f:
    reader = csv.DictReader(f)
    for i,row in enumerate(reader):
        if i==10:break
        for k,v in row.items():
            print (k,":",v)

artist : ABBA
song : Ahe's My Kind Of Girl
link : /a/abba/ahes+my+kind+of+girl_20598417.html
text : Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  
  
And when we go for a walk in the park  
And she holds me and squeezes my hand  
We'll go on walking for hours and talking  
About all the things that we plan  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?


artist : ABBA
song : Andante, Andante
link : /a/abba/andante+andante_20002708.html
text : Take it easy with me, please  
Touch me gently l

### Load all dataset in memory

**Disclaimer**: in the following we limit the number of songs to make sure we have enough memory and reasonable running times. It's clear that 
**the more songs the more interesting the result**.

In [6]:
def load_data(filename, max_songs = 5000):
    rows = []

    with open(filename, newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            rows += [ [x for x in row.values()] ]
            if len(rows)>=max_songs:
                break
    return rows

raw_dataset = load_data(songs_file)

print ("artist:", raw_dataset[1][0])
print ("title:",  raw_dataset[1][1])
print ("lyrics:", raw_dataset[1][-1])

artist: ABBA
title: Andante, Andante
lyrics: Take it easy with me, please  
Touch me gently like a summer evening breeze  
Take your time, make it slow  
Andante, Andante  
Just let the feeling grow  
  
Make your fingers soft and light  
Let your body be the velvet of the night  
Touch my soul, you know how  
Andante, Andante  
Go slowly with me now  
  
I'm your music  
(I am your music and I am your song)  
I'm your song  
(I am your music and I am your song)  
Play me time and time again and make me strong  
(Play me again 'cause you're making me strong)  
Make me sing, make me sound  
(You make me sing and you make me)  
Andante, Andante  
Tread lightly on my ground  
Andante, Andante  
Oh please don't let me down  
  
There's a shimmer in your eyes  
Like the feeling of a thousand butterflies  
Please don't talk, go on, play  
Andante, Andante  
And let me float away  
  
I'm your music  
(I am your music and I am your song)  
I'm your song  
(I am your music and I am your song) 

#### Search for "Pink" by Aerosmith

Try with another song of your choice.

In [7]:
for i,row in enumerate(raw_dataset):
    if "Pink" in row[1]:
        print(i,row[0],row[1])

183 Aerosmith Pink
780 Ariana Grande Pink Champagne
2126 Cake Pretty Pink Ribbon


In [8]:
print (raw_dataset[183][-1])

Pink, it's my new obsession, yeah  
Pink, it's not even a question  
Pink, on the lips of your lover  
'Cause pink is the love you discover  
Pink, as the bing on your cherry  
Pink, 'cause you are so very  
Pink, it's the color of passion  
  
'Cause today it just goes with the fashion  
Pink, it was love at first sight  
Yeah pink, when I turn out the light  
And pink gets me high as a kite  
And I think everything is going to be alright  
No matter what we do tonight  
  
You could be my flamingo  
'Cause pink, it's a new kinda lingo  
Pink, like a deco umbrella  
Ffff, it's kink that you don't ever tell her  
Yeah, pink, it was love at first sight  
Then pink when I turn out the light  
Yeah, pink gets me high as a kite  
And I think everything is going to be alright  
No matter what we do tonight  
  
Yeah,  
I, want to be your lover  
Ffff, I I wanna wrap you in rubber  
And it's pink as the sheets that we lay on  
'Cause pink, It's my favorite crayon  
  
Yeah  
Pink, it was lov

In [9]:
for i,(a,t,u,l) in enumerate(raw_dataset):
    if "Pink" in t:
        print(i,a,t)

183 Aerosmith Pink
780 Ariana Grande Pink Champagne
2126 Cake Pretty Pink Ribbon


In [10]:
# This is the query song
query_id = 183
skip = [] # if any, put covers here !

In [11]:
print ( raw_dataset[query_id][-1])

Pink, it's my new obsession, yeah  
Pink, it's not even a question  
Pink, on the lips of your lover  
'Cause pink is the love you discover  
Pink, as the bing on your cherry  
Pink, 'cause you are so very  
Pink, it's the color of passion  
  
'Cause today it just goes with the fashion  
Pink, it was love at first sight  
Yeah pink, when I turn out the light  
And pink gets me high as a kite  
And I think everything is going to be alright  
No matter what we do tonight  
  
You could be my flamingo  
'Cause pink, it's a new kinda lingo  
Pink, like a deco umbrella  
Ffff, it's kink that you don't ever tell her  
Yeah, pink, it was love at first sight  
Then pink when I turn out the light  
Yeah, pink gets me high as a kite  
And I think everything is going to be alright  
No matter what we do tonight  
  
Yeah,  
I, want to be your lover  
Ffff, I I wanna wrap you in rubber  
And it's pink as the sheets that we lay on  
'Cause pink, It's my favorite crayon  
  
Yeah  
Pink, it was lov

## How to find the most similar song?

We need two ingredients:
 - define what is a song?
 - define when two songs are similar
 
better:
 - define a **representation** for the song
 - define a **similarity** function 


Representation and similarity are two key ingredients in several data mining tasks, e.g., collaborative filtering, clustering, etc. Beyond the limitations of the example below, you should first design a suitable similarity function, and then find a good representation to implement such similarity function.

## Option 1

 - A song is a **set of words**
 - Similarity is given by the **number of shared words**

#### Compute the set of words of each song

In [12]:
print (raw_dataset[0][0], raw_dataset[0][1])

ABBA Ahe's My Kind Of Girl


In [13]:
print (raw_dataset[0][-1])

Look at her face, it's a wonderful face  
And it means something special to me  
Look at the way that she smiles when she sees me  
How lucky can one fellow be?  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?  
  
And when we go for a walk in the park  
And she holds me and squeezes my hand  
We'll go on walking for hours and talking  
About all the things that we plan  
  
She's just my kind of girl, she makes me feel fine  
Who could ever believe that she could be mine?  
She's just my kind of girl, without her I'm blue  
And if she ever leaves me what could I do, what could I do?




In [14]:
# Python allows sets !
print ( set(raw_dataset[0][-1].split()) )

{"it's", 'on', 'wonderful', 'things', 'what', 'plan', 'when', 'fellow', "We'll", 'it', 'feel', 'blue', 'leaves', 'at', 'that', 'talking', 'to', 'if', 'without', 'holds', 'Look', 'face', 'of', 'can', 'hours', 'her', 'smiles', 'face,', 'kind', 'mine?', 'could', 'ever', 'me', 'walk', 'squeezes', 'walking', 'she', 'for', 'one', 'the', "She's", 'all', 'we', 'a', 'go', 'How', 'just', 'do?', 'in', 'fine', 'do,', 'means', 'Who', 'girl,', 'believe', 'park', 'About', 'my', 'I', 'and', 'hand', 'something', 'way', 'special', 'be', 'lucky', 'And', 'be?', "I'm", 'sees', 'makes'}


In [15]:
def get_words (songs):
    songs_words = []
    for s in songs:
        lyrics = s[-1]         # this is a string
        words = lyrics.split() # this is a list
        words = set(words)     # create a set
        songs_words += [words] # append words to the list
    return songs_words


lyrics_word_split = get_words(raw_dataset)

print ( lyrics_word_split[0:2] )

[{"it's", 'on', 'wonderful', 'things', 'what', 'plan', 'when', 'fellow', "We'll", 'it', 'feel', 'blue', 'leaves', 'at', 'that', 'talking', 'to', 'if', 'without', 'holds', 'Look', 'face', 'of', 'can', 'hours', 'her', 'smiles', 'face,', 'kind', 'mine?', 'could', 'ever', 'me', 'walk', 'squeezes', 'walking', 'she', 'for', 'one', 'the', "She's", 'all', 'we', 'a', 'go', 'How', 'just', 'do?', 'in', 'fine', 'do,', 'means', 'Who', 'girl,', 'believe', 'park', 'About', 'my', 'I', 'and', 'hand', 'something', 'way', 'special', 'be', 'lucky', 'And', 'be?', "I'm", 'sees', 'makes'}, {'on', 'eyes', 'Like', 'it', 'your', 'make', 'me)', 'night', 'butterflies', 'slow', 'know', 'strong)', 'time,', 'Let', 'down', "don't", "There's", 'a', 'float', 'my', '(You', 'Andante', 'like', 'summer', 'song', 'me,', 'now', 'how', 'fingers', 'am', 'again', 'of', 'gently', 'Take', 'the', 'let', 'talk,', 'body', 'away', 'grow', 'time', 'Play', 'ground', 'I', 'music', 'be', 'strong', 'And', '(I', 'shimmer', 'feeling', 'play

In [16]:
# As above, but in one line
def get_words (songs):
    return [ set(s[-1].split()) for s in songs ]

lyrics_word_split = get_words(raw_dataset)

print ( lyrics_word_split[0] )

{"it's", 'on', 'wonderful', 'things', 'what', 'plan', 'when', 'fellow', "We'll", 'it', 'feel', 'blue', 'leaves', 'at', 'that', 'talking', 'to', 'if', 'without', 'holds', 'Look', 'face', 'of', 'can', 'hours', 'her', 'smiles', 'face,', 'kind', 'mine?', 'could', 'ever', 'me', 'walk', 'squeezes', 'walking', 'she', 'for', 'one', 'the', "She's", 'all', 'we', 'a', 'go', 'How', 'just', 'do?', 'in', 'fine', 'do,', 'means', 'Who', 'girl,', 'believe', 'park', 'About', 'my', 'I', 'and', 'hand', 'something', 'way', 'special', 'be', 'lucky', 'And', 'be?', "I'm", 'sees', 'makes'}


In [17]:
print (lyrics_word_split[query_id])

{'on', 'yeah', 'passion', 'Yeah', 'pink,', 'what', 'going', 'but', "'Cause", 'it', 'fashion', 'sight', 'to', 'your', 'is', 'want', 'flamingo', 'today', 'No', 'not', "don't", 'umbrella', 'lay', 'we', 'a', 'wrap', 'gets', 'just', 'discover', 'as', 'think', 'my', 'like', 'Then', 'favorite', 'sight,', "it's", 'You', 'bing', 'kite', 'of', 'cherry', 'could', 'obsession,', 'the', 'pink', 'out', 'I', 'turn', 'be', 'wanna', 'Ffff,', 'lover', 'question', 'high', 'And', 'alright', 'Pink,', 'goes', 'even', 'quite', "It's", 'you', 'me', 'red', 'crayon', 'lips', 'very', 'kink', 'color', 'was', "'cause", 'kinda', 'lingo', 'love', 'when', 'light', 'rubber', 'matter', 'at', 'that', 'do', 'deco', 'with', 'her', 'Yeah,', 'tell', 'so', 'are', 'ever', 'everything', 'sheets', 'in', 'first', 'tonight', 'I,', 'new'}


#### Find the most similar

In [18]:
# Test similarity
print ( lyrics_word_split[0] & lyrics_word_split[query_id])

{"it's", 'on', 'what', 'when', 'it', 'at', 'that', 'to', 'of', 'her', 'could', 'ever', 'me', 'the', 'we', 'a', 'just', 'in', 'my', 'I', 'be', 'And'}


In [19]:
print ( len(lyrics_word_split[0] & lyrics_word_split[query_id]) )

22


In [20]:
def most_similar_by_words(s, songs, skip_list):
    most_similar = None
    largest_similarity = 0.0
    
    for s_id, s_text in enumerate(songs):
        
        if s_id == query_id: continue
        if s_id in skip_list: continue
        
        # compute number of common words
        sim = len(s_text & songs[s])      

        if sim>=largest_similarity:
            most_similar = s_id
            largest_similarity = sim
    
    return most_similar, largest_similarity

sim_id, sim_value = most_similar_by_words(query_id, lyrics_word_split, skip)

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

Most similar song is: 151
Similarity is: 45
Artist: Aerosmith
Title: Fever
Lyrics: I got a rip in my shoes  
And a hole in my brand new shoes  
I got a Margarita nose  
And a breath full of Mad Dog Booze  
I got the fever, fever, fever, fever  
Yeah, they threw me outta jail  
I tell ya it ain't fair  
I tried to kiss the judge  
From the electrica' chair  
Yeah we're all here  
'Cause we're not all there tonight  
The guitar's cranked  
And the bass man's blown a fuse  
And when the whole gang bangs  
Then what is your excuse?  
I got the fever, fever, fever, fever  
Fever gives you lust with an appetite  
It hits you like the fangs  
From a rattlesnake bite  
Yeah we're all here  
'Cause we're not all there tonight  
We can't run away from trouble  
There ain't no place that far  
But if we do it right at the speed of light  
There's the backseat of my car - caviar  
I was feelin' so high I forgot what day  
Now I'm feeling low down  
Even slow feels way to fast  
And now the booze d

Can we do it in one line of code?

In [25]:
# a small check
print ( max([1,2,3]) )
print ( max([(2,1),(2,2),(1,3)]) )

#List comprehensions
#Prima metto espressione e poi un'iterazione
nuova_l = [i**2 for i in range(10)]
print (nuova_l)

3
(2, 2)
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [26]:
## Exercise

def most_similar_by_words(s, songs, skiplist):
    most_similar = max( [ (len(s_text & songs[s]), s_id) 
                             for s_id, s_text in enumerate(songs) 
                             if s_id not in skiplist ]   )
    return most_similar[1], most_similar[0]

    
sim_id, sim_value = most_similar_by_words(query_id, lyrics_word_split, 
                                          set(skip+[query_id]))

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

Most similar song is: 151
Similarity is: 45
Artist: Aerosmith
Title: Fever
Lyrics: I got a rip in my shoes  
And a hole in my brand new shoes  
I got a Margarita nose  
And a breath full of Mad Dog Booze  
I got the fever, fever, fever, fever  
Yeah, they threw me outta jail  
I tell ya it ain't fair  
I tried to kiss the judge  
From the electrica' chair  
Yeah we're all here  
'Cause we're not all there tonight  
The guitar's cranked  
And the bass man's blown a fuse  
And when the whole gang bangs  
Then what is your excuse?  
I got the fever, fever, fever, fever  
Fever gives you lust with an appetite  
It hits you like the fangs  
From a rattlesnake bite  
Yeah we're all here  
'Cause we're not all there tonight  
We can't run away from trouble  
There ain't no place that far  
But if we do it right at the speed of light  
There's the backseat of my car - caviar  
I was feelin' so high I forgot what day  
Now I'm feeling low down  
Even slow feels way to fast  
And now the booze d

just too many words in this song ?

Some exercises:
 - longest song
 - shortest song
 - song with most unique terms
 - song with least unique terms
 - artist with most songs
 - artist with most unique terms
 - how many unique terms in all songs

In [76]:
# Longest song
def longest_song(songs):
    lyrics_list = [s[-1].split() for s in songs]
    return len(max(lyrics_list, key= len)), lyrics_list.index(max(lyrics_list, key= len))

longest_lenght, longest_id = longest_song(raw_dataset)

print ("Longest song is:", longest_id)
print ("Lenght is:", longest_lenght)
print ("Artist:", raw_dataset[longest_id][0])
print ("Title:", raw_dataset[longest_id][1])
print ("Lyrics:", raw_dataset[longest_id][-1])

# Shortest song
def shortest_song(songs):
    lyrics_list = [s[-1].split() for s in songs]
    return len(min(lyrics_list, key=len)), lyrics_list.index(min(lyrics_list, key= len))

shortest_lenght, shortest_id = shortest_song(raw_dataset)

print ("Shortest song is:", shortest_id)
print ("Lenght is:", shortest_lenght)
print ("Artist:", raw_dataset[shortest_id][0])
print ("Title:", raw_dataset[shortest_id][1])
print ("Lyrics:", raw_dataset[shortest_id][-1])

# Song with most unique terms

def terms_frequency(songs):
    terms_frequency = {}
    for s in songs:
        terms_list = s[-1].split()
        for l in terms_list:
            terms_frequency[l] = terms_frequency.get(l,0) + 1
    return (terms_frequency)

def most_unique_song(songs, terms_frequency):
    song_unique_terms = []
    for s in songs:
        terms_list = s[-1].split()
        i = 0
        for l in terms_list:
            if terms_frequency[l] == 1: i += 1
        song_unique_terms+= [i]
    return max(song_unique_terms), song_unique_terms.index(max(song_unique_terms))
        

number_unique_terms, unique_song_id = most_unique_song(raw_dataset,terms_frequency(raw_dataset))

print ("Song with most unique terms is:", unique_song_id)
print ("Number of unique terms is:", number_unique_terms)
print ("Artist:", raw_dataset[unique_song_id][0])
print ("Title:", raw_dataset[unique_song_id][1])
print ("Lyrics:", raw_dataset[unique_song_id][-1])

# Song with least unique terms

def least_unique_song(songs, terms_frequency):
    song_unique_terms = []
    for s in songs:
        terms_list = s[-1].split()
        i = 0
        for l in terms_list:
            if terms_frequency[l] == 1: i += 1
        song_unique_terms+= [i]
    return min(song_unique_terms), song_unique_terms.index(min(song_unique_terms))
        

number_unique_terms, least_unique_song_id = least_unique_song(raw_dataset,terms_frequency(raw_dataset))

print ("Song with least unique terms is:", least_unique_song_id)
print ("Number of unique terms is:", number_unique_terms)
print ("Artist:", raw_dataset[least_unique_song_id][0])
print ("Title:", raw_dataset[least_unique_song_id][1])
print ("Lyrics:", raw_dataset[least_unique_song_id][-1])

# Artist with most songs

def artist_most_songs(songs):
    artists = [s[0] for s in songs]
    songs_per_artist = {s:artists.count(s) for s in artists}
    return max(songs_per_artist, key = songs_per_artist.get), max(songs_per_artist.values())

artist_most_id, n_songs = artist_most_songs(raw_dataset)

print ("Artist with most songs is:", artist_most_id)
print ("Number of songs is:", n_songs)

print()
# Artist with most unique terms

def artist_most_unique_terms(songs, terms_frequency):
    artists = {s[0]:0 for s in songs}
    for s in songs:
        terms_list = s[-1].split()
        i = 0
        for l in terms_list:
            if terms_frequency[l] == 1: artists[s[0]] += 1
        
    return max(artists, key = artists.get), max(artists.values())

artist_id, n_terms = artist_most_unique_terms(raw_dataset,terms_frequency(raw_dataset))

print ("Artist with most unique terms is:", artist_id)
print ("Number of terms is:", n_terms)
print()

def n_unique_terms(terms_frequency):
    return list(terms_frequency.values()).count(1)

print ("Unique terms number:", n_unique_terms(terms_frequency(raw_dataset)))




Longest song is: 2079
Lenght is: 727
Artist: Bruno Mars
Title: Nothin On You
Lyrics: Beautiful girls all over the world  
I could be chasing but my time would be wasted  
They got nothin' on you baby  
Nothin' on you baby  
  
They might say hi and I might say hey  
But you shouldn't worry about what they say  
'Cause they got nothin' on you baby  
Nothin' on you baby  
  
No' no' no' nothin' on you babe  
No' no' nothin' on you  
I know you feel where I'm coming from  
Regardless of the things in my past that I've done  
Most of it really was for the hell of the fun  
On the carousel so around I spun (spun)  
With no directions just tryna get some (some)  
Tryna chase skirts, living in the summer sun (sun)  
An' so I lost more than I had ever won  
And honestly I ended up with none  
  
There's so much nonsense  
It's on my conscience  
I'm thinking baby I should get it out  
And I don't wanna sound redundant  
But I was wondering, if there was something that you wanna know  
(That yo

Artist with most songs is: Conway Twitty
Number of songs is: 121

Artist with most unique terms is: Chris Brown
Number of terms is: 781

Unique terms number: 20086


# Jaccard similarity


$$
J(A,B) = \frac{|A\cap B|}{|A \cup B|}
$$

In [96]:
#più documenti sonomlunghi più aumenta denominatore, più diminuisce valore: cardinalità intersez/ cardinalità unione

def jaccard(a,b):
    return len(a & b) / len( a | b)

print ( jaccard( set([1,2,3]), set([2,3,4])))

0.5


In [28]:
def most_similar_jaccard(s, songs, skiplist):
    most_similar = max( [ (jaccard(s_text, songs[s]), s_id) 
                             for s_id, s_text in enumerate(songs) 
                             if s_id not in skiplist ]   )
    return most_similar[1], most_similar[0]

sim_id, sim_value = most_similar_jaccard(query_id, lyrics_word_split, 
                                         set(skip+[query_id]))

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

Most similar song is: 2390
Similarity is: 0.21379310344827587
Artist: Chaka Khan
Title: Jigsaw
Lyrics: Jigsaw - Puzzle  
Jigsaw - Puzzle  
  
Your love is like a maze  
I can't get through to you  
You keep me in a daze  
What's a girl to do with you?  
  
Loving you is like a puzzle  
Just when I think that this is it  
You shake me up  
The pieces just don't fit  
It's like a ...  
Jigsaw - Puzzle  
  
I lay my heart out on the table  
I let you know my every move  
But darlin' you are so unstable  
Your needle never lifts the groove  
  
Now order may not be your nature  
But I think that we could work out fine  
If you would only trace the dotted line  
  
Jigsaw - Puzzle  
Jigsaw - Puzzle




In [29]:
print (raw_dataset[query_id][0])
print (raw_dataset[query_id][1])
print (raw_dataset[query_id][-1])

Aerosmith
Pink
Pink, it's my new obsession, yeah  
Pink, it's not even a question  
Pink, on the lips of your lover  
'Cause pink is the love you discover  
Pink, as the bing on your cherry  
Pink, 'cause you are so very  
Pink, it's the color of passion  
  
'Cause today it just goes with the fashion  
Pink, it was love at first sight  
Yeah pink, when I turn out the light  
And pink gets me high as a kite  
And I think everything is going to be alright  
No matter what we do tonight  
  
You could be my flamingo  
'Cause pink, it's a new kinda lingo  
Pink, like a deco umbrella  
Ffff, it's kink that you don't ever tell her  
Yeah, pink, it was love at first sight  
Then pink when I turn out the light  
Yeah, pink gets me high as a kite  
And I think everything is going to be alright  
No matter what we do tonight  
  
Yeah,  
I, want to be your lover  
Ffff, I I wanna wrap you in rubber  
And it's pink as the sheets that we lay on  
'Cause pink, It's my favorite crayon  
  
Yeah  
P

In [30]:
# non abbiamo considerato punteggiatura e maiuscole minuscole, si fa tokenizzazione

lyrics_word_split[query_id]

{"'Cause",
 "'cause",
 'And',
 'Ffff,',
 'I',
 'I,',
 "It's",
 'No',
 'Pink,',
 'Then',
 'Yeah',
 'Yeah,',
 'You',
 'a',
 'alright',
 'are',
 'as',
 'at',
 'be',
 'bing',
 'but',
 'cherry',
 'color',
 'could',
 'crayon',
 'deco',
 'discover',
 'do',
 "don't",
 'even',
 'ever',
 'everything',
 'fashion',
 'favorite',
 'first',
 'flamingo',
 'gets',
 'goes',
 'going',
 'her',
 'high',
 'in',
 'is',
 'it',
 "it's",
 'just',
 'kinda',
 'kink',
 'kite',
 'lay',
 'light',
 'like',
 'lingo',
 'lips',
 'love',
 'lover',
 'matter',
 'me',
 'my',
 'new',
 'not',
 'obsession,',
 'of',
 'on',
 'out',
 'passion',
 'pink',
 'pink,',
 'question',
 'quite',
 'red',
 'rubber',
 'sheets',
 'sight',
 'sight,',
 'so',
 'tell',
 'that',
 'the',
 'think',
 'to',
 'today',
 'tonight',
 'turn',
 'umbrella',
 'very',
 'wanna',
 'want',
 'was',
 'we',
 'what',
 'when',
 'with',
 'wrap',
 'yeah',
 'you',
 'your'}

# Attempt 1: Tokenization

Text Processing library: https://textblob.readthedocs.io/en/dev/

In [77]:
from textblob import Sentence, TextBlob

# if the above command is not working
# execute below

In [49]:
# linux
import sys
!{sys.executable} -m pip install TextBlob

# windows try
#pip install TextBlob

import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ricca\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [80]:
TextBlob(raw_dataset[query_id][-1])

TextBlob(raw_dataset[query_id][-1]).lower()

TextBlob(raw_dataset[query_id][-1]).lower().words

WordList(['pink', 'it', "'s", 'my', 'new', 'obsession', 'yeah', 'pink', 'it', "'s", 'not', 'even', 'a', 'question', 'pink', 'on', 'the', 'lips', 'of', 'your', 'lover', "'cause", 'pink', 'is', 'the', 'love', 'you', 'discover', 'pink', 'as', 'the', 'bing', 'on', 'your', 'cherry', 'pink', "'cause", 'you', 'are', 'so', 'very', 'pink', 'it', "'s", 'the', 'color', 'of', 'passion', "'cause", 'today', 'it', 'just', 'goes', 'with', 'the', 'fashion', 'pink', 'it', 'was', 'love', 'at', 'first', 'sight', 'yeah', 'pink', 'when', 'i', 'turn', 'out', 'the', 'light', 'and', 'pink', 'gets', 'me', 'high', 'as', 'a', 'kite', 'and', 'i', 'think', 'everything', 'is', 'going', 'to', 'be', 'alright', 'no', 'matter', 'what', 'we', 'do', 'tonight', 'you', 'could', 'be', 'my', 'flamingo', "'cause", 'pink', 'it', "'s", 'a', 'new', 'kinda', 'lingo', 'pink', 'like', 'a', 'deco', 'umbrella', 'ffff', 'it', "'s", 'kink', 'that', 'you', 'do', "n't", 'ever', 'tell', 'her', 'yeah', 'pink', 'it', 'was', 'love', 'at', 'fi

In [81]:
def get_tokens (songs):
    return [ set(TextBlob(song[-1]).lower().words) for song in songs ]

lyrics_tokens = get_tokens(raw_dataset)

In [82]:
print ( sorted(lyrics_word_split[query_id] ) )
print ()
print ( sorted(lyrics_tokens[query_id] ) )

["'Cause", "'cause", 'And', 'Ffff,', 'I', 'I,', "It's", 'No', 'Pink,', 'Then', 'Yeah', 'Yeah,', 'You', 'a', 'alright', 'are', 'as', 'at', 'be', 'bing', 'but', 'cherry', 'color', 'could', 'crayon', 'deco', 'discover', 'do', "don't", 'even', 'ever', 'everything', 'fashion', 'favorite', 'first', 'flamingo', 'gets', 'goes', 'going', 'her', 'high', 'in', 'is', 'it', "it's", 'just', 'kinda', 'kink', 'kite', 'lay', 'light', 'like', 'lingo', 'lips', 'love', 'lover', 'matter', 'me', 'my', 'new', 'not', 'obsession,', 'of', 'on', 'out', 'passion', 'pink', 'pink,', 'question', 'quite', 'red', 'rubber', 'sheets', 'sight', 'sight,', 'so', 'tell', 'that', 'the', 'think', 'to', 'today', 'tonight', 'turn', 'umbrella', 'very', 'wanna', 'want', 'was', 'we', 'what', 'when', 'with', 'wrap', 'yeah', 'you', 'your']

["'cause", "'s", 'a', 'alright', 'and', 'are', 'as', 'at', 'be', 'bing', 'but', 'cherry', 'color', 'could', 'crayon', 'deco', 'discover', 'do', 'even', 'ever', 'everything', 'fashion', 'favorite'

In [53]:
sim_id, sim_value = most_similar_jaccard(query_id, lyrics_tokens, 
                                         set(skip+[query_id]))

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

Most similar song is: 156
Similarity is: 0.25308641975308643
Artist: Aerosmith
Title: Lay It Down
Lyrics: Ruby red... her lips were on fire  
A do me with a kiss, if you please  
Tell me what your sweet heart desires  
Tell me how you want it to be  
  
'Cause if it's love you want  
Then you won't mind a little tenderness  
That sometimes is so hard to find  
  
(Lay it down)  
Lay it down  
Make it alright  
(Lay it down)  
Lay it down  
I'll hold you so tight  
(Lay it down)  
Oh...before the morning light  
It's gonna be alright  
  
Oh...lay it down  
Come and lay it down tonight  
  
Tell me how you feel when we make love  
Tell me is it real or just make believe  
You will never know what'chor made of  
'Til you open up your heart to receive  
'Cause if the love you got that same old crime  
We're talkin' tenderness that's so hard to find  
And I'm gettin' behind you  
  
(Lay it down)  
Lay it down  
Make it alright  
(Lay it down)  
A lay it down  
I'll hold you so tight  
(La

# Attempt 2: Stemming, Lemmatization

Stemming refers to the removal of prefix/suffixes: `being` -> `be`, `was`->`was` 

Lemming refers to the identification of the "origin" of a word: `being` -> `be`, `was`->`be`


The task is quite difficult, there might be errors anyway ...

In [54]:
print ( sorted(set(TextBlob(raw_dataset[query_id][-1]).lower().words.stem() ) ) )

["'caus", "'s", 'a', 'alright', 'and', 'are', 'as', 'at', 'be', 'bing', 'but', 'cherri', 'color', 'could', 'crayon', 'deco', 'discov', 'do', 'even', 'ever', 'everyth', 'fashion', 'favorit', 'ffff', 'first', 'flamingo', 'get', 'go', 'goe', 'her', 'high', 'i', 'in', 'is', 'it', 'just', 'kinda', 'kink', 'kite', 'lay', 'light', 'like', 'lingo', 'lip', 'love', 'lover', 'matter', 'me', 'my', "n't", 'na', 'new', 'no', 'not', 'obsess', 'of', 'on', 'out', 'passion', 'pink', 'question', 'quit', 'red', 'rubber', 'sheet', 'sight', 'so', 'tell', 'that', 'the', 'then', 'think', 'to', 'today', 'tonight', 'turn', 'umbrella', 'veri', 'wa', 'wan', 'want', 'we', 'what', 'when', 'with', 'wrap', 'yeah', 'you', 'your']


In [91]:
def get_stems (songs):
    return [ set(TextBlob(song[-1]).lower().words.stem()) for song in raw_dataset ]

lyrics_stems = get_stems(raw_dataset)

In [92]:
print ( sorted(lyrics_stems[query_id] ) )

["'caus", "'s", 'a', 'alright', 'and', 'are', 'as', 'at', 'be', 'bing', 'but', 'cherri', 'color', 'could', 'crayon', 'deco', 'discov', 'do', 'even', 'ever', 'everyth', 'fashion', 'favorit', 'ffff', 'first', 'flamingo', 'get', 'go', 'goe', 'her', 'high', 'i', 'in', 'is', 'it', 'just', 'kinda', 'kink', 'kite', 'lay', 'light', 'like', 'lingo', 'lip', 'love', 'lover', 'matter', 'me', 'my', "n't", 'na', 'new', 'no', 'not', 'obsess', 'of', 'on', 'out', 'passion', 'pink', 'question', 'quit', 'red', 'rubber', 'sheet', 'sight', 'so', 'tell', 'that', 'the', 'then', 'think', 'to', 'today', 'tonight', 'turn', 'umbrella', 'veri', 'wa', 'wan', 'want', 'we', 'what', 'when', 'with', 'wrap', 'yeah', 'you', 'your']


In [94]:
sim_id, sim_value = most_similar_jaccard(query_id, lyrics_stems, set(skip+[query_id]))

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

NameError: name 'most_similar_jaccard' is not defined

In [ ]:
print ( lyrics_stems[sim_id] )

In [59]:
print ( sorted( lyrics_stems[sim_id] & lyrics_stems[query_id]) )

["'s", 'a', 'are', 'be', 'but', 'could', 'do', 'get', 'i', 'in', 'is', 'it', 'just', 'lay', 'like', 'love', 'me', 'my', "n't", 'not', 'on', 'out', 'so', 'that', 'the', 'think', 'to', 'we', 'what', 'when', 'with', 'you', 'your']


Exercise:
 - find the pair of songs whose similarity that was most affected by the stemming

In [126]:
def most_affected_pair(lyrics_tokens, lyrics_stems):
    pair = ()
    maxval = 0
    for i, (t1, s1) in enumerate(zip(lyrics_tokens, lyrics_stems)):
        for j, (t2, s2) in enumerate(zip(lyrics_tokens, lyrics_stems), i + 1):
            if abs(jaccard(t1,t2) - jaccard(s1,s2)) > maxval:
                maxval = abs(jaccard(t1,t2) - jaccard(s1,s2))
                pair = (i,j)
    return pair

song1, song2 = most_affected_pair(lyrics_tokens, lyrics_stems)

print ("Most affected pair is is:", raw_dataset[song1][1], raw_dataset[song2][1])

KeyboardInterrupt: 

# Attempt 3: Term Frequency and Inverse Document Frequency


We want to take into account the number of occurences of terms. Therefore we need to change our representation:
 - A song is vector of occurences
 - Similarity is ...
 
 
To do so, we use a large matrix #songs x #terms.

### Build a vector space

In [129]:
## Exercise: find the lexicon


lexicon = set([])
for song in lyrics_stems:
    lexicon |= set(song)

print (len(lexicon))

16919
{'accid', 'def', 'thrub', 'chopper', 'bumpi', 'rican', 'phycadel', 'unsouffl', 'bleedin', 'norweigian', 'funhous', 'gees', 'yoke', 'gram', 'diet', 'often', 'lead', 'hilaga', 'shineth', 'excess', 'zee', 'dispers', 'overcomin', 'gard', 'freak', 'lev', 'hang', 'klan', 'voi', 'deisel', 'afriad', 'unwind', 'evergreen', 'joanna', 'pleas', 'ojo', 'squirm', 'groovi', 'brotha', 'chateaux', 'dreamer', 'sanctuari', 'useless', 'cra', 'keepin', 'breezier', 'peaux', "'one", 'regal-eagl', 'float', 'unravel', 'vasser', 'dirti', 'jamai', 'swiftli', 'paralyz', 'spinman', 'harshli', 'ur', 'cipher', 'reactor', 'grindin', 'looki', 'squander', 'gatekeep', 'eternidad', 'dit', 'chooglin', 'lavish', 'doctor', 'tear', 'nap', 'osaka', 'marmalad', 'vista', 'scatter', 'murki', "should't", 'whoop-ee-doo', 'gnaw', 'dustpan', 'nameless', 'bo-bo-bo-boy', 'threaten', 'cosh', 'jojo', 'caramel', 'stamped', 'pina', 'wife', 'bobsl', 'methadon', 'bride', 'mirth', 'precipit', 'undoubtedli', 'franticli', "cow'stal", 'ba

In [135]:
lexicon_map     = {term:id for id,term in enumerate(lexicon)}
lexicon_rev_map = {id:term for term,id in lexicon_map.items()}

In [62]:
lexicon_map['pink']

16462

In [65]:
lexicon_rev_map[16462]

'pink'

#### Use numpy for matrix-based computations

http://www.numpy.org/

In [130]:
import numpy as np

#Matrice riempita di zeri

lyrics_vector_space = np.zeros((len(lyrics_stems), len(lexicon)))


In [131]:
lyrics_vector_space.shape

(5000, 16919)

In [132]:
lyrics_vector_space.dtype

dtype('float64')

In [136]:
def get_vector_space (songs, lex_map):
    m = np.zeros((len(songs), len(lex_map)))

    for song_id, (s,t,l,song_text) in enumerate(songs):
        for stem in TextBlob(song_text).lower().words.stem():
            if stem in lex_map:
                term_id = lex_map[stem]
                m[song_id,term_id] += 1.0
    
    return m

lyrics_vector_space = get_vector_space(raw_dataset, lexicon_map)

In [68]:
lyrics_vector_space[query_id]

array([0., 0., 0., ..., 0., 0., 0.])

In [69]:
sum(lyrics_vector_space[query_id])

236.0

In [70]:
lyrics_vector_space[query_id]!=0

array([False, False, False, ..., False, False, False])

In [71]:
sum(lyrics_vector_space[query_id]!=0)

89

In [72]:
len(lyrics_stems[query_id])

89

Good, numbers look consistent!

#### What kind of similarity we can use ?

#### This is euclidean ...

In [73]:
# let's try with euclidean

a = np.array([1,2,3])
b = np.array([1,5,7])

print (a-b)

[ 0 -3 -4]


In [74]:
print ( (a-b)**2 )

[ 0  9 16]


In [75]:
print ( np.sum((a-b)**2) )

25


In [76]:
print ( np.sqrt(np.sum((a-b)**2)) )

5.0


In [77]:
def euclidean(a,b):
    return np.sqrt( np.sum((a-b)**2.0) )

In [78]:
def most_similar_euclidean(s, songs, skiplist):
    num_songs, num_terms = songs.shape
    most_similar = min( [ (euclidean(songs[s], songs[s_id]), s_id) 
                             for s_id in range(num_songs) 
                             if s_id not in skiplist ]   )
    return most_similar[1], most_similar[0]

sim_id, sim_value = most_similar_euclidean(query_id, lyrics_vector_space, 
                                           set(skip+[query_id]))

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

Most similar song is: 2548
Similarity is: 30.643106892089126
Artist: Cher
Title: Fires Of Eden
Lyrics: It's not over til it's over  
I heard someone say  
Must be a whisper in the wind  
'Cause you're too far away  
  
But in my restless sleep  
I could swear I saw you next to me  
Saying , oh I'm coming home  
You'll never spend another night alone  
  
[Chorus]  
Remember when love was innocent  
There was never a better time  
But you know those fires of Eden  
Still burn in this heart of mine  
  
The morning's uncertain  
It's a nervous day  
And I look for a reason  
Why I should feel this way  
  
I hear a voice run before  
Drifting through my open door  
Saying it's alright  
We're going to light those flames tonight  
  
[Chorus]  
  
And don't you wonder  
How we drifted so far  
When we belong to each other  
We were miles apart  
  
And there's a place  
That was meant for the two of us  
And when you touch the embers  
You feel my love  
As strong as it ever was  
  
[Cho

In [79]:
print ( sorted(lyrics_stems[sim_id] & lyrics_stems[query_id]) )

["'caus", "'s", 'a', 'alright', 'and', 'as', 'be', 'but', 'could', 'do', 'ever', 'go', 'i', 'in', 'it', 'light', 'love', 'me', 'my', "n't", 'not', 'of', 'so', 'that', 'the', 'to', 'tonight', 'wa', 'we', 'when', 'you']


#### Space for proposals

#### What we need is Cosine Similarity


$$
\cos(A,B)= \frac{A \cdot B}{\|A\| \|B\|} = \frac{\sum_i A_i B_i}{\sqrt{\sum_i A_i^2}\sqrt{\sum_i B_i^2}}
$$

In [80]:
def cosine(a,b):
    return np.dot(a,b)/( np.sqrt( np.sum(a**2.0) ) * np.sqrt( np.sum(b**2.0) ) )

In [ ]:
cosine( np.array([1,0]), np.array([0,1]) )

In [ ]:
cosine( np.array([1,0]), np.array([2,0]) )

In [81]:
def most_similar_cosine(s, songs, skiplist):
    num_songs, num_terms = songs.shape
    most_similar = max( [ (cosine(songs[s], songs[s_id]), s_id) 
                             for s_id in range(num_songs) 
                             if s_id not in skiplist ]   )
    return most_similar[1], most_similar[0]

sim_id, sim_value = most_similar_cosine(query_id, lyrics_vector_space, set(skip+[query_id]))

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

Most similar song is: 99
Similarity is: 0.5622402881617116
Artist: ABBA
Title: The Winner Takes It All
Lyrics: I don't want to talk  
About the things we've gone through  
Though it's hurting me  
Now it's history  
I've played all my cards  
And that's what you've done too  
Nothing more to say  
No more ace to play  
  
The winner takes it all  
The loser standing small  
Beside the victory  
That's my destiny  
  
I was in your arms  
Thinking I belonged there  
I figured it made sense  
Building me a fence  
Building me a home  
Thinking I'd be strong there  
But I was a fool  
Playing by the rules  
  
The gods may throw a dice  
Their minds as cold as ice  
And someone way down here  
Loses someone dear  
The winner takes it all  
The loser has to fall  
It's simple and it's plain  
Why should I complain.  
  
But tell me does she kiss  
Like I used to kiss you?  
Does it feel the same  
When she calls your name?  
Somewhere deep inside  
You must know I miss you  
But what can I

#### Let's investigate the words that contribute most

In [ ]:
# normalized prod instead of dot product

def norm_prod(a,b):
    return a*b/( np.sqrt( np.sum(a**2.0) ) * np.sqrt( np.sum(b**2.0) ) )

p = norm_prod(lyrics_vector_space[sim_id], lyrics_vector_space[query_id])
print (len(p), p)

In [ ]:
idx = np.argsort(p)
print (idx)

In [ ]:
prova = np.array([1,3,5,4,2])

In [ ]:
np.argsort(prova)

In [ ]:
prova[ np.argsort(prova) ]

In [ ]:
prova[ np.argsort(prova)[::-1] ]

In [ ]:
p[idx]

In [ ]:
p[idx[::-1]]

In [ ]:
for term_id in np.argsort(idx)[::-1]:
    if p[term_id]>0:
        print (lexicon_rev_map[term_id])

### Inverse document frequency

Inverse document frequency approximates the specificity of a term in a given document collection.
IDF is defined as follows:

$$
idf(t) = \ln \frac{N_{docs}}{df(t)}
$$

where $N_{docs}$ is the number of documents in the collection and ${df(t)}$ is the number of documents containing the term $t$.

IDF is used to discount frequent terms.

The weight of a term for a document is thus defined as $tf(t)\cdot idf(t)$.

In [ ]:
m_sum = np.sum(lyrics_vector_space)

In [ ]:
print (m_sum)

In [ ]:
lyrics_vector_space.shape

In [ ]:
m_sum = np.sum(lyrics_vector_space, axis=1)
print (m_sum)
print (m_sum.shape)

In [ ]:
m_sum = np.sum(lyrics_vector_space, axis=0)
print (m_sum)
print (m_sum.shape)

In [ ]:
m_sum = np.sum(lyrics_vector_space>0, axis=0)
print (m_sum)
print (m_sum.shape)

In [ ]:
# let's do a small test

A = np.array([ [1,2,3], [4,5,6] ])
b = np.array([1,2,3])

print (A)
print (b)
print (A*b)

In [ ]:
print( np.sum(A/b>2, axis=0) )

In [ ]:
num_docs, _ = lyrics_vector_space.shape

In [ ]:
lyrics_tdif = lyrics_vector_space * np.log( num_docs/m_sum)

In [ ]:
def most_similar_cosine(s, songs, skiplist):
    num_songs, num_terms = songs.shape
    most_similar = max( [ (cosine(songs[s], songs[s_id]), s_id) 
                             for s_id in range(num_songs) 
                             if s_id not in skiplist ]   )
    return most_similar[1], most_similar[0]

sim_id, sim_value = most_similar_cosine(query_id, lyrics_tdif, set(skip+[query_id]))

print ("Most similar song is:", sim_id)
print ("Similarity is:", sim_value)
print ("Artist:", raw_dataset[sim_id][0])
print ("Title:", raw_dataset[sim_id][1])
print ("Lyrics:", raw_dataset[sim_id][-1])

In [ ]:
# normalized prod instead of dot product

def norm_prod(a,b):
    return a*b/( np.sqrt( np.sum(a**2.0) ) * np.sqrt( np.sum(b**2.0) ) )

p = norm_prod(lyrics_vector_space[sim_id], lyrics_vector_space[query_id])

idx = np.argsort(p)

for term_id in np.argsort(idx)[::-1]:
    if p[term_id]>0:
        print (lexicon_rev_map[term_id])

#### How do you like it?

# Is this enough ?

## Exercises:

 - remove stop-words, too frequent words, unusual/infrequent words
 - Find the most original song:
   - most distant on average?
   - most distant from the 10 closest ones?
 - [advanced] filter using additional functionalities of textblob, e.g., lemmatize rather than stem, use only nouns, anything else comes up to your mind

In [ ]:
from nltk.corpus import stopwords
print(sorted(stopwords.words('english')))

## Good to know about TextBlob

### Get Sentiment/Polarity of first sentence

TextBlob provides a polarity score, i.e., a number between -1 (negative) and 1 (positive).

In [ ]:
from textblob import Sentence

In [ ]:
Sentence("I love playing tennis!").polarity

In [ ]:
Sentence("I hate running!").polarity

### Translation

In [ ]:
chinese_blob = TextBlob(u"美丽优于丑陋")
chinese_blob.translate(from_lang="zh-CN", to='en')

In [ ]:
eng_blob = TextBlob("Beauty is better than ugly")
eng_blob.translate(from_lang="en", to='it')

# References

 - **Introduction to Information Retrieval**. Manning, Raghavan, Schütze. Cambridge University Press. 2008.
   - Sections 2.1, 2.2, 6.2, 6.3
   - Download: https://nlp.stanford.edu/IR-book/information-retrieval-book.html
 - **Web Data Mining** 2nd edition. Liu. Springer. 2011.
   - Sections 6.2.1, 6.2.2, 6.5
 - **Mining of Massive Datasets**. Leskovec, Rajaraman, Ullman. Cambridge University Press. 2014.
   - Sections 3.1
   - Download: http://www.mmds.org/